# Proyecto Final Analítica de Datos
##### Hecho por: 
##### * Juan Sebastián Clavijo Martínez (jclavijomartinez@gmail.com) - PUJ - Ing. en redes y telecomunicaciones
##### * Santiago Camacho (santiagocamachov@javeriana.edu.co ) - PUJ - Ing. de sistemas
##### * Juan Pablo González (gonzalez-juanp@javeriana.edu.co) - PUJ - Ing. de sistemas <br />
**Profesor: Jhon Corredor**<br />
Fecha: 14-11-2023<br />
Notas: <br />
Dataset: Datos de nacimiento de los pueblos de antioquia y chocó del 2009 a 2019 obtenidos de la pagína del DANE e información sobre areas deforestadas en el chocó obtenido de datos.gov.co

In [ ]:
#Se importan las bibliotecas pertinentes para el desarrollo del proyecto
import pandas as pd
import numpy as np
import time ##Para medir rendimiento de los modelos
import matplotlib.pyplot as plt 
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.preprocessing import OrdinalEncoder
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn import metrics

In [ ]:
#Se crean dos dataframes, uno para cada set de datos. El primero corresponde al set de datos de areas deforestadas en el depaertamento del chocó en Colombia. El segundo corresponde a los datos de nacimiento de algunos municipios en Colombia. 
areas_deforestadas=pd.read_csv("https://raw.githubusercontent.com/jclavijomartinez/analiticaDeDatos2330/master/datos%20proyecto%20final/AREAS_DEFORESTADAS_CHOCO.csv")
# Para el dataframe de nacimientos se carga el archivo CSV especificando el delimitador como punto y coma ya que en el archivo CSV no se utiliza la coma como delimitador sino el ;
nacimientos = pd.read_csv("https://raw.githubusercontent.com/jclavijomartinez/analiticaDeDatos2330/master/datos%20proyecto%20final/finaldatosnacimientos.csv", sep=';')


In [ ]:
#Se imprime la información del dataframe de areas deforestadas del chocó.
areas_deforestadas.info()

In [ ]:
#Se imprime la información del dataframde de nacimientos en los municipios del país.
nacimientos.info()

##### OBSERVACIÓN: Se evidencia que en ambos dataframes los valores de las columnas no son los mismos, lo que indica que hay datos duplicados y/o faltantes. A continuación se realizará la limpieza de los datasets teniendo como objetivo eliminar los dataos faltantes y los datos duplicados. 

In [ ]:
##Cuenta de datas null, etc... para el dataframe de nacimientos     
desaparecidos = len(nacimientos)-len(nacimientos.dropna())

print('Cantidad de observaciones con Datos NaN', desaparecidos)
print('Cantidad de datos duplicados', nacimientos.duplicated().sum())

In [ ]:
##Cuenta de datas null, etc... para el dataframe de areas deforestadas
desaparecidos = len(areas_deforestadas)-len(areas_deforestadas.dropna())

print('Cantidad de observaciones con Datos NaN', desaparecidos)
print('Cantidad de datos duplicados', areas_deforestadas.duplicated().sum())

In [ ]:
## Se eliminan los datos Null y Duplicados para ambos dataframes
areas_deforestadas.dropna(inplace=True)
areas_deforestadas.drop_duplicates(inplace = True)
nacimientos.dropna(inplace=True)
nacimientos.drop_duplicates(inplace=True)
#Se reinicia el indice por los eliminados (duplicados)
areas_deforestadas.reset_index(drop = True, inplace = True)
nacimientos.reset_index(drop = True, inplace = True)

In [ ]:
#Se imprime la nueva información del dataset de nacimientos por municipio
nacimientos.info()

In [ ]:
#Se imprime la nueva información del dataset de areas deforestadas
areas_deforestadas.info()

##### OBSERVACIÓN: Se evidencia que se eliminaron los registros duplicados y valores faltantes para ambos datasets. Es decir, todas las columnas de ambos datasets tienen exactamente el mismo número de registros. 

##### nota: se cambiarán los nombres de algunas columnas de los datasets para que cumplan con el estándar PEP8

In [ ]:
#Se cambian los nombres de ciertas columnas para que cumplan el estándar PEP8
areas_deforestadas = areas_deforestadas.rename(columns={'TIPO GEOMETRIA': 'TIPO_GEOMETRIA'})
nacimientos = nacimientos.rename(columns={'total H': 'total_H'})
nacimientos = nacimientos.rename(columns={'total M': 'total_M'})
nacimientos = nacimientos.rename(columns={'total indet': 'total_indet'})
nacimientos = nacimientos.rename(columns={'cabecera municipal H': 'cabecera_municipal_H'})
nacimientos = nacimientos.rename(columns={'cabecera municipal M': 'cabecera_municipal_M'})
nacimientos = nacimientos.rename(columns={'cebecera municipal indet': 'cabecera_municipal_indet'})
nacimientos = nacimientos.rename(columns={'centro poblado H': 'centro_poblado_H'})
nacimientos = nacimientos.rename(columns={'centro poblado M': 'centro_poblado_M'})
nacimientos = nacimientos.rename(columns={'centro poblado indet': 'centro_poblado_indet'})
nacimientos = nacimientos.rename(columns={'rural disperso H': 'rural_disperso_H'})
nacimientos = nacimientos.rename(columns={'rural disperso M': 'rural_disperso_M'})
nacimientos = nacimientos.rename(columns={'rural disperso indet': 'rural_disperso_indet'})
nacimientos = nacimientos.rename(columns={'sin info H': 'sin_info_H'})
nacimientos = nacimientos.rename(columns={'sin info M': 'sin_info_M'})
nacimientos = nacimientos.rename(columns={'sin info indet': 'sin_info_indet'})




In [ ]:
#Se imprime la nueva información del dataset de areas deforestadas
areas_deforestadas.info()

In [ ]:
#Se imprime la nueva información del dataset de nacimientos
nacimientos.info()

Se imprimen los primeros 15 registros de cada dataframe

In [ ]:
nacimientos.head(5)

In [ ]:
areas_deforestadas.head(5)

In [ ]:
# ya que los dfs están listos para ser usados, 
# se crea uno nuevo con la información de los pueblos a ser evaluados, se va a usar primero arboles de desicion como modelo de ML, 
# se vizualiza la informacion del nuevo df con los datos refinados
municipios_interes=['ZARAGOZA','NECHÍ','NÓVITA']
df_seleccion = nacimientos[nacimientos['municipio'].isin(municipios_interes)]
df_seleccion.head(5)

In [ ]:
# Seleccionamos las características y el objetivo
caracteristicas = ['departamento', 'total_H', 'total_M', 'total_indet']
objetivo = 'total'

# Creamos el DataFrame con las características y el objetivo
df_seleccion = df_seleccion[caracteristicas + [objetivo]]

# Convertimos 'departamento' a números usando OrdinalEncoder
encoder = OrdinalEncoder()
df_seleccion['departamento'] = encoder.fit_transform(df_seleccion[['departamento']])

# Se asignan características y objetivo
X = df_seleccion[caracteristicas]
y = df_seleccion[objetivo]

In [ ]:
# Dividimos el conjunto en entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
# Creamos el modelo de árbol de decisión
modelo_arbol = DecisionTreeClassifier(random_state=42)

# Entrenamos el modelo
modelo_arbol.fit(X_train, y_train)

# Reemplazamos las comas en la columna '0,00' y convertimos a float
df_seleccion['0,00'] = df_seleccion['0,00'].str.replace(',', '.').astype(float)

# Creamos el modelo de árbol de decisión
modelo_arbol = DecisionTreeClassifier(random_state=42)

# Entrenamos el modelo
modelo_arbol.fit(X_train, y_train)

In [ ]:
# Hacemos predicciones en el conjunto de prueba
predicciones = modelo_arbol.predict(X_test)

In [ ]:
# Evaluamos la precisión
precision = accuracy_score(y_test, predicciones)
print(f"Precisión del modelo: {precision}")

# Mostramos la matriz de confusión
matriz_confusion = confusion_matrix(y_test, predicciones)
print("Matriz de Confusión:")
print(matriz_confusion)

# Mostramos el reporte de clasificación
reporte_clasificacion = classification_report(y_test, predicciones)
print("Reporte de Clasificación:")
print(reporte_clasificacion)

In [ ]:
# Definir una función de conversión
def convertir_a_float(value):
    try:
        # Utilizar pd.to_numeric para manejar posibles errores de conversión
        return pd.to_numeric(value.replace(',', '.'), errors='coerce')
    except ValueError:
        return np.nan  # Manejar errores de conversión como NaN

# Aplicar la función a la columna 'total_H'
nacimientos['total_H'] = nacimientos['total_H'].apply(convertir_a_float)

# Manejar valores nulos después de la conversión
nacimientos = nacimientos.dropna()


In [ ]:
#Se dividen los datos en conjuntos de entrenamiento y prueba

# Seleccionar características y variable dependiente
X = nacimientos[['total_H', 'total_M', 'centro_poblado_H']]  
y = nacimientos['total']  

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)



In [ ]:
# Función para convertir y manejar comas
# Función mejorada para convertir y manejar comas
def convertir_a_float(valor):
    try:
        # Reemplazar comas y puntos al mismo tiempo
        return float(str(valor).replace(',', '').replace('.', ''))
    except ValueError:
        return valor  # Si ya es un float, devolverlo sin cambios

# Aplicar la función a las columnas relevantes
nacimientos['total_H'] = nacimientos['total_H'].apply(convertir_a_float)
nacimientos['total_M'] = nacimientos['total_M'].apply(convertir_a_float)
nacimientos['centro_poblado_H'] = nacimientos['centro_poblado_H'].apply(convertir_a_float)
nacimientos['total'] = nacimientos['total'].apply(convertir_a_float)

# Manejar valores nulos después de la conversión
nacimientos = nacimientos.dropna()




In [ ]:
# Mostrar información sobre las columnas de X_train
print(X_train.info())



<class 'pandas.core.frame.DataFrame'>
Int64Index: 610 entries, 336 to 102
Data columns (total 3 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   total_H           610 non-null    float64
 1   total_M           610 non-null    float64
 2   centro_poblado_H  610 non-null    float64
dtypes: float64(3)
memory usage: 19.1 KB
None


In [ ]:
#Entrenar el modelo de regresión lineal múltiple
model = LinearRegression()
model.fit(X_train, y_train)


Out[70]: LinearRegression()

In [ ]:
y_pred = model.predict(X_test)

# Evaluar el modelo
print('Error absoluto medio:', metrics.mean_absolute_error(y_test, y_pred))
print('Error cuadrático medio:', metrics.mean_squared_error(y_test, y_pred))
print('Raíz cuadrada del error cuadrático medio:', metrics.mean_squared_error(y_test, y_pred)**0.5)


Error absoluto medio: 108.91797149938323
Error cuadrático medio: 72754.75862586104
Raíz cuadrada del error cuadrático medio: 269.7309003912252


## Conclusiones

1. Error absoluto: El error absoluto es de aproximadamente 108.92. Esto significa que, en promedio, las predicciones del modelo tienen un error absoluto de alrededor de 108.92 unidades con respecto a los valores reales de la variable 'total'
2. Error Cuadrático Medio: El error cuadrático medio es de aproximadamente 72754.76. Este valor indica que el modelo tiene un error cuadrático medio de alrededor de 72754.76 unidades al cuadrado con respecto a los valores reales. Es importante tener en cuenta que el MSE castiga más los errores grandes que el MAE.
3. Raíz Cuadrada del Error Cuadrático Medio: La raíz cuadrada del error cuadrático medio  es de aproximadamente 269.73. Esta métrica es simplemente la raíz cuadrada del MSE y proporciona una medida de la dispersión de los errores. En este caso, el RMSE indica que, en promedio, las predicciones tienen un error de alrededor de 269.73 unidades con respecto a los valores reales.

- Conclusión:

  - El MAE de 108.92 sugiere que, en promedio, las predicciones están bastante cerca de los valores reales, pero aún hay margen para mejorar.

  - El MSE de 72754.76 indica que hay algunos errores significativos en las predicciones, ya que los errores grandes tienen un impacto considerable en esta métrica.

  - El RMSE de 269.73 proporciona una medida más interpretable del error y sugiere que, en promedio, las predicciones están a unos 269.73 unidades de los valores reales.